In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#         cv2.imshow('img',img)
#         cv2.waitKey(500)

cv2.destroyAllWindows()

In [3]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [25]:
sample_checkboard_img = cv2.imread(images[0])
undist_checkboard = cv2.undistort(sample_checkboard_img, mtx, dist, None, mtx)
fig, axes= plt.subplots(1,2,figsize=(20,8))
axes[0].imshow(sample_checkboard_img)
axes[1].imshow(undist_checkboard)
plt.savefig('output_images/undistorted.png')

In [26]:
input_img = cv2.imread('./test_images/test5.jpg')
input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(input_img)

In [27]:
undist = cv2.undistort(input_img, mtx, dist, None, mtx)
plt.imshow(undist)

In [7]:
def calculateTransformMatrix(img,reverse = False):
    h,w,_ = img.shape   
    src = np.float32([[w*0.45,h*0.64],[w*0.16,h],[w*0.88,h],[w*0.55,h*0.64]])
    dst = np.float32([[w*0.25,0],[w*0.25,h],[w*0.75,h],[w*0.75,0]])
#     src = np.float32([[w*0.12,h],[w*0.46,h*62],[w*0.55,h*0.62],[w*0.97,h]])
#     dst = np.float32([[w*0.2,h],[w*0.2,w],[w*0.75,w],[w*0.75,0]])
    if reverse:
        M = cv2.getPerspectiveTransform(dst,src)
    else:
        M = cv2.getPerspectiveTransform(src,dst)
    return M

In [8]:
# src = np.float32([[150,720],[590,450],[700,450],[1250,720]])
# dst = np.float32([[200,720],[200,0],[980,0],[980,720]])
M = calculateTransformMatrix(undist)
corrected = cv2.warpPerspective(undist,M,(undist.shape[1], undist.shape[0]))
plt.imshow(corrected)
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [19]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
#     Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    result = np.bitwise_and(sxbinary,s_binary)
#     Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return result
thresh = pipeline(corrected, s_thresh=(10, 255), sx_thresh=(20, 100))
fig, axes= plt.subplots(1,3,figsize=(20,8))
axes[0].imshow(input_img)
axes[1].imshow(corrected)
axes[2].imshow(thresh)
plt.savefig('output_images/thresh.png')

In [10]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
            
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy>=win_y_low) & (nonzeroy<win_y_high) & (nonzerox>=win_xleft_low) & (nonzerox<win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy>=win_y_low) & (nonzeroy<win_y_high) & (nonzerox>=win_xright_low) & (nonzerox<win_xright_high)).nonzero()[0]
        # Append these indices to the lishigh
        right_lane_inds.append(good_right_inds)
        left_lane_inds.append(good_left_inds)
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds)>minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds)>minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        pass # Remove this when you add your function

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

In [11]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    #Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    #Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

In [12]:
def search_new_lanes(binary_warped):
    #Fit a second order polynomial to each using `np.polyfit` #
    leftx, lefty, rightx, righty, lane_img = find_lane_pixels(binary_warped)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fit = np.polyfit(lefty,leftx,2)
        right_fit = np.polyfit(righty,rightx,2)
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        lanes_found = True
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        lanes_found = False
        
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img = np.zeros_like(lane_img)
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    zeros = np.array(np.where(binary_warped.T >= 0))
    zeros_x = zeros[0]
    zeros_y = zeros[1]
    region = []
    if(lanes_found):
        lane_region = ((left_fit[0]*zeros_y**2 + left_fit[1]*zeros_y + left_fit[2]<zeros_x)&(right_fit[0]*zeros_y**2 + right_fit[1]*zeros_y + right_fit[2]>zeros_x)).nonzero()[0]
        region.append(lane_region)
        out_img[zeros_y[np.concatenate(region)],zeros_x[np.concatenate(region)]] = [0,255,0]
    
        curve_rad = measure_curvature_real(ploty,left_fit,right_fit)

        y_vehicle = binary_warped.shape[0]
        vehicle_offset = xm_per_pix*(left_fit[0]*y_vehicle**2 + left_fit[1]*y_vehicle + left_fit[2] + right_fit[0]*y_vehicle**2 + right_fit[1]*y_vehicle + right_fit[2] - binary_warped.shape[1])/2
        return out_img, curve_rad, vehicle_offset, left_fit, right_fit, lanes_found
    else:
        return out_img, 0, 0, -1, -1, lanes_found

In [13]:
def search_around_poly(binary_warped, left_fit,right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    if(left_fit!=None and right_fit!=None):
        if abs(left_fit[0]-right_fit[0])<0.1 and abs(left_fit[1]-right_fit[1])<0.1 :
            margin = 100

            # Grab activated pixels
            nonzero = binary_warped.nonzero()
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            ### Set the area of search based on activated x-values ###
            ### within the +/- margin of our polynomial function ###
            ### Hint: consider the window areas for the similarly named variables ###
            ### in the previous quiz, but change the windows to our new search area ###
            left_lane_inds = ((left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - nonzerox >=-margin) &
                (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] - nonzerox <=margin)).nonzero()[0]
            right_lane_inds = ((right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - nonzerox >=-margin) &
                (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] - nonzerox <=margin)).nonzero()[0]
            # Again, extract left and right line pixel positions
            leftx = nonzerox[left_lane_inds]
            lefty = nonzeroy[left_lane_inds] 
            rightx = nonzerox[right_lane_inds]
            righty = nonzeroy[right_lane_inds]

            out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
            # Fit new polynomials
            left_fit = np.polyfit(lefty,leftx,2)
            right_fit = np.polyfit(righty,rightx,2)
    else:
        try:
            leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
            left_fit = np.polyfit(lefty,leftx,2)
            right_fit = np.polyfit(righty,rightx,2)
        except:
            return out_img, None, None, 0, 0
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    zeros = np.array(np.where(binary_warped.T >= 0))
    zeros_x = np.array(zeros[0])
    zeros_y = np.array(zeros[1])
    lane_region = ((left_fit[0]*zeros_y**2 + left_fit[1]*zeros_y + left_fit[2]<zeros_x)&(right_fit[0]*zeros_y**2 + right_fit[1]*zeros_y + right_fit[2]>zeros_x)).nonzero()[0]
    out_img[zeros_y[np.concatenate([lane_region])],zeros_x[np.concatenate([lane_region])]] = [0,255,0]
    curve_rad = measure_curvature_real(ploty,left_fit,right_fit)
    y_vehicle = binary_warped.shape[0]
    vehicle_offset = xm_per_pix*(left_fit[0]*y_vehicle**2 + left_fit[1]*y_vehicle + left_fit[2] + right_fit[0]*y_vehicle**2 + right_fit[1]*y_vehicle + right_fit[2] - binary_warped.shape[1])/2
        
    return out_img, left_fit, right_fit, curve_rad, vehicle_offset

In [14]:
def measure_curvature_real(ploty, left_fit_cr, right_fit_cr):  
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    ##### Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    curve_rad = int(min(left_curverad, right_curverad))

    return curve_rad

In [15]:
def addText(img,curve_rad,vehicle_offset):
    cv2.putText(img,'Radius of Curvature = '+str(curve_rad)+'(m)', (120,50),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    if vehicle_offset>=0:
        cv2.putText(img,'Vehicle is %.2f'%vehicle_offset+'m left of center', (120,110),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)
    else:
        cv2.putText(img,'Vehicle is %.2f'%-vehicle_offset+'m right of center', (120,110),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2) 
    return img

In [29]:
lane_region, curve_rad, vehicle_offset, left_fit, right_fit, lanes_found = search_new_lanes(thresh)
M_inv = calculateTransformMatrix(input_img, True)
lanes_unwarped = cv2.warpPerspective(lane_region,M_inv,(undist.shape[1], undist.shape[0]))
annotated_lanes = cv2.addWeighted(undist,0.8,lanes_unwarped,0.2,1)
result = addText(annotated_lanes,curve_rad, vehicle_offset)
fig, axes= plt.subplots(1,3,figsize=(20,8))
axes[0].imshow(corrected)
axes[1].imshow(thresh)
axes[2].imshow(lane_region)
plt.savefig('output_images/lane_region.png')
fig = plt.plot()
plt.imshow(result)
plt.savefig('output_images/result.png')

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
left_fit,right_fit, M, M_inv = None, None, None, None
def process_image(frame):
    global left_fit,right_fit, M, M_inv
    if M is None:
        M = calculateTransformMatrix(frame)
        M_inv = calculateTransformMatrix(frame,reverse = True)
    undist = cv2.undistort(frame, mtx, dist, None, mtx)
    corrected = cv2.warpPerspective(undist,M,(undist.shape[1], undist.shape[0]))
    thresh = pipeline(corrected, s_thresh=(100, 255), sx_thresh=(10, 100))
    try:
        lane_region, left_fit, right_fit, curve_rad, vehicle_offset = search_around_poly(thresh,left_fit,right_fit)
    except:
        lane_region, left_fit, right_fit, curve_rad, vehicle_offset = search_around_poly(thresh,None,None)
    
    lanes_unwarped = cv2.warpPerspective(lane_region,M_inv,(undist.shape[1], undist.shape[0]))
    annotated_lanes = cv2.addWeighted(undist,0.5,lanes_unwarped,0.5,1)
    if curve_rad:
        result = addText(annotated_lanes,curve_rad, vehicle_offset)
    else:
        result = annotated_lanes
    return result

In [ ]:
white_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("project_video.mp4").subclip(23,28)
clip1 = VideoFileClip("project_video.mp4")
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [18]:
plt.imshow(thresh)